In [ ]:
import matplotlib.pyplot as plt
from numpy.random import seed
seed(101)
import tensorflow as tf
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np
#import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation ,MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam


import os
import cv2

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input
#from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [ ]:
#Upload Dataset.csv
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
df= pd.read_csv('train.csv')
df.head()

,ID,Text,Class
0,7303,"""Please enable removing language code from the...",1
1,5258,"""Note: in your .csproj file, there is a Suppor...",0
2,6218,"""Wich means the new version not fully replaced...",0
3,3076,"""Some of my users will still receive the old x...",0
4,6508,"""The store randomly gives the old xap or the n...",0


In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#with_lemmatizer_and_Stemmer_bs
from nltk.corpus import stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z ]')
STOPWORDS = set(stopwords.words('english'))

import string
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

def clean_text_ls(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    #text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r";", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer('english')
    #stem_words = [stemmer.stem(word) for word in text if len(word)>1]
    #lem_words = [lemmatizer.lemmatize(word) for word in stem_words if len(word)>1]
    lem_words = [lemmatizer.lemmatize(word) for word in text if len(word)>1]
    text = " ".join(lem_words)
    return text

In [ ]:
'''#plain_text_pre_processing
from nltk.corpus import stopwords
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

STOPWORDS = set(stopwords.words('english'))

def clean_text_p(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r";", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)# remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text'''

In [ ]:
df['Text'] = df['Text'].map(lambda x: clean_text_ls(x))
df['Text'] = df['Text'].str.replace('\d+', '')
df['Text'] = df['Text'].str.strip().str.split(' ')

In [ ]:
def print_plot(index):
    example = df[df.index == index][['Text', 'Class']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Class:', example[1])
print_plot(10)

['the', 'store', 'will', 'randomly', 'deliver', 'the', 'old', 'wrong', 'version', 'of', 'your', 'xap']
Class: 0


In [ ]:
df.loc[0,'Text']

['please',
 'enable',
 'removing',
 'language',
 'code',
 'from',
 'the',
 'dev',
 'center',
 'language',
 'history',
 'for',
 'example',
 'if',
 'you',
 'ever',
 'selected',
 'ru',
 'and',
 'ru',
 'ru',
 'laguages',
 'and',
 'you',
 'published',
 'this',
 'xap',
 'to',
 'the',
 'store',
 'then',
 'it',
 'cause',
 'tile',
 'localization',
 'to',
 'show',
 'the',
 'en',
 'u',
 'default',
 'tile',
 'localization',
 'which',
 'is',
 'bad']

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 8305 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['Text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (8500, 250)


In [ ]:
Y = df['Class']
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (8500,)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6800, 250) (6800,)
(1700, 250) (1700,)


In [ ]:
from sklearn.metrics import f1_score
import keras.backend as K
from sklearn.manifold import TSNE

In [ ]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-11-18 18:12:12--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-11-18 18:12:12--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-11-18 18:12:13--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
!ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip  train.csv
glove.6B.200d.txt  glove.6B.50d.txt   sample_data


In [ ]:
#import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SpatialDropout1D,LSTM,Dense,Input,Bidirectional,Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.layers.embeddings import Embedding

In [ ]:
embeddings_index = dict()
f = open('glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
embedding_matrix = np.zeros((MAX_NB_WORDS, 200))
for word, index in tokenizer.word_index.items():
    if index > MAX_NB_WORDS - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [ ]:
## create model
model_glove = Sequential()
model_glove.add(Embedding(MAX_NB_WORDS, 200, input_length=250, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(512, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(128))
model_glove.add(Dense(16,activation='relu'))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss=f1_loss, optimizer='adam', metrics=['accuracy',f1])
model_glove.summary()
## Fit train data



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 200)          10000000  
_________________________________________________________________
dropout (Dropout)            (None, 250, 200)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 246, 512)          512512    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 61, 512)           0         
_________________________________________________________________
lstm (LSTM)                

In [ ]:
check_glove = ModelCheckpoint('modelA3_glv-2016B2A80592G.hdf5', verbose=0,monitor='val_acc', save_best_only=True, save_weights_only=False, mode='auto')
model_glove.fit(X_train, Y_train, epochs=160, batch_size=1024,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss', patience=200, min_delta=0.0001)])

Train on 5440 samples, validate on 1360 samples
Epoch 1/160
5440/5440 [==============================] - 1s 234us/sample - loss: 0.0553 - acc: 0.9732 - f1: 0.9470 - val_loss: 0.2806 - val_acc: 0.8662 - val_f1: 0.7187
Epoch 2/160
5440/5440 [==============================] - 1s 221us/sample - loss: 0.0575 - acc: 0.9722 - f1: 0.9439 - val_loss: 0.3059 - val_acc: 0.8625 - val_f1: 0.6977
Epoch 3/160
5440/5440 [==============================] - 1s 221us/sample - loss: 0.0542 - acc: 0.9741 - f1: 0.9458 - val_loss: 0.2827 - val_acc: 0.8669 - val_f1: 0.7134
Epoch 4/160
5440/5440 [==============================] - 1s 219us/sample - loss: 0.0602 - acc: 0.9708 - f1: 0.9433 - val_loss: 0.3019 - val_acc: 0.8669 - val_f1: 0.7254
Epoch 5/160
5440/5440 [==============================] - 1s 219us/sample - loss: 0.0610 - acc: 0.9711 - f1: 0.9397 - val_loss: 0.2825 - val_acc: 0.8566 - val_f1: 0.7178
Epoch 6/160
5440/5440 [==============================] - 1s 238us/sample - loss: 0.0614 - acc: 0.9699 - f1:

In [ ]:
model_glove.save('modelA3_glv-2016B2A80592G.hdf5')
 

In [ ]:
from google.colab import files
files.download(r'modelA3_glv-2016B2A80592G.hdf5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 39698, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [ ]:
from IPython.display import FileLink
FileLink(r'modelA3_glv-2016B2A80592G.hdf5')

/content/modelA3_glv-2016B2A80592G.hdf5

In [ ]:
model_glove.load_weights('modelA3_glv-2016B2A80592G.hdf5')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
ts = pd.read_csv('test.csv')
ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 2 columns):
ID      1425 non-null int64
Text    1425 non-null object
dtypes: int64(1), object(1)
memory usage: 22.4+ KB


In [ ]:
ts['Text'] = ts['Text'].map(lambda x: clean_text_ls(x))
ts['Text'] = ts['Text'].str.replace('\d+', '')
ts['Text'] = ts['Text'].str.strip().str.split(' ')
ts.head()

,ID,Text
0,9348,"[this, would, enable, live, traffic, aware, apps]"
1,9393,"[please, try, other, formatting, like, bold, i..."
2,8557,"[since, computer, were, invented, to, save, ti..."
3,9288,"[allow, rearranging, if, the, user, want, to, ..."
4,9231,"[add, simd, instruction, for, better, use, of,..."


In [ ]:
def print_plot(index):
    example = ts[ts.index == index][['Text']].values[0]
    if len(example) > 0:
        print(example[0])
print_plot(1000)


['multi', 'select', 'on', 'zune', 'to', 'delete', 'album', 'or', 'to', 'create', 'playlist']


In [ ]:
Xt = tokenizer.texts_to_sequences(ts['Text'].values)
Xt = pad_sequences(Xt, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', Xt.shape)

Shape of data tensor: (1425, 250)


In [ ]:
pred=model_glove.predict(Xt)
#pred=model.predict(Xt)

In [ ]:
pred
#ts['ID']

array([[2.9385090e-05],
       [9.9960887e-01],
       [2.9802322e-08],
       ...,
       [1.4391929e-02],
       [9.8316407e-01],
       [1.0000000e+00]], dtype=float32)

In [ ]:
class_ts=[]


In [ ]:
for i in range(len(pred)):
  class_ts.append(int(np.round(pred[i])))

In [ ]:
class_ts

[0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
ts['ID'].shape
c_t=np.asarray(class_ts, dtype = None, order = None)


In [ ]:
data=({'ID':ts['ID'],'Class':class_ts})
#subm=pd.DataFrame({'ID':ts['ID'],'Class':pred})

In [ ]:
subm = pd.DataFrame(data)

In [ ]:
subm.head()

,ID,Class
0,9348,0
1,9393,1
2,8557,0
3,9288,1
4,9231,1


In [ ]:
subm.to_csv(r'submission_a3_2016B2A80592G.csv',index=False)

In [ ]:
from google.colab import files
files.download(r'submission_a3_2016B2A80592G.csv') 
